In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import sentencepiece

c:\Users\sukan\PycharmProjects\Polent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
# Step 1: Install Necessary Libraries
# Ensure you have the required libraries installed:
# !pip install transformers accelerate bitsandbytes

# Step 2: Set Up BitsAndBytes Configuration
# Configure the model to load in 4-bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [4]:

# Step 3: Load the Mistral 7B Model and Tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)


Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.50s/it]


In [5]:

# Step 4: Define the Prompt for Directed Sentiment Analysis
def create_sentiment_prompt(sentence, person_a, person_b):
    prompt = (
        f"Analyze the sentiment between {person_a} and {person_b} in the following sentence:\n"
        f"\"{sentence}\"\n"
        f"Please specify if the sentiment is positive, negative, or neutral."
        f"If the sentiment goes from {person_a} towards {person_b}, specify 1 otherwise 0. "
    )
    return prompt


In [6]:

# Step 4: Define the Prompt for Directed Sentiment Analysis
def create_sentiment_prompt(sentence, person_a, person_b):
    prompt = (
        f"You are given a sentence where a person has directed sentiment towards another person"
        f"\"{sentence}\"\n"
        f"Please specify: \"\n0 if the sentiment between {person_a} and {person_b} is neutral."
        f"\"\n 1 if {person_a} feels positively about {person_b}"
        f"\"\n 2 if {person_b} feels positively about {person_a}"
        f"\"\n 3 if {person_a} feels negatively about {person_b}"
        f"\"\n 4 if {person_b} feels negatively about {person_a}"
    )
    return prompt


In [7]:

# Step 5: Generate Sentiment Analysis Using the Model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

def analyze_sentiment(sentence, person_a, person_b):
    prompt = create_sentiment_prompt(sentence, person_a, person_b)
    response = pipe(
        prompt,
        do_sample=True,
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
    )
    sentiment = response[0]['generated_text'].strip()
    return sentiment


Device set to use cuda:0


In [8]:


sentence = "That early morning call marked a new phase in the collaborative relationship that [Newsom] has built with [Trump] behind the scenes"
person_a = "Newsom"
person_b = "Trump"

sentiment = analyze_sentiment(sentence, person_a, person_b)
print(f"Sentiment expressed by {person_a} towards {person_b}: {sentiment}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentiment expressed by Newsom towards Trump: You are given a sentence where a person has directed sentiment towards another person"That early morning call marked a new phase in the collaborative relationship that [Newsom] has built with [Trump] behind the scenes"
Please specify: "
0 if the sentiment between Newsom and Trump is neutral."
 1 if Newsom feels positively about Trump"
 2 if Trump feels positively about Newsom"
 3 if Newsom feels negatively about Trump"
 4 if Trump feels negatively about Newsom"
 The sentiment in this sentence is not specified. However, the phrase "a new phase in the collaborative relationship" implies that there is some level of positive sentiment between Newsom and Trump. So, I would classify this as a sentiment
